In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Config

In [52]:
test_split = 0.05
validation_split = 0.05
learning_rate = 1e-3
input_size = 300
hidden_size = 225
bottleneck_size = 200
epochs = 5
batch_size = 128
use_hidden_layer = False
denoising = True
experiment_name = "autoencoder_300to200_v2"
dataset = "data/embeddings/base/clipped.glove.6B.300d.txt"

In [53]:
gpu = torch.cuda.is_available()
device = torch.device("cuda" if gpu else "cpu")

In [54]:
print(gpu, device)

False cpu


# Load & Prepare Embeddings for Training

In [7]:
words = []
vectors = []
with open(dataset, "r", encoding='utf8') as fp:
    for line in fp:
        line = line.split()
        word = line[0]
        vector = np.asarray(line[1:], 'float32')
        words.append(word)
        vectors.append(vector)
vectors = torch.from_numpy(np.asarray(vectors))

In [8]:
test_split = int(test_split * len(words))
validation_split = int(validation_split* len(words))

In [9]:
print(len(words), vectors.shape)
print(test_split, validation_split)

400000 torch.Size([400000, 300])
20000 20000


In [10]:
train_vectors, test_vectors, train_words, test_words = train_test_split(
    vectors, words, test_size=test_split, random_state=seed
)

In [11]:
train_vectors, validation_vectors, train_words, validation_words = train_test_split(
    train_vectors, train_words, test_size=validation_split, random_state=seed
)

In [12]:
train_vectors.shape, validation_vectors.shape, test_vectors.shape

(torch.Size([360000, 300]), torch.Size([20000, 300]), torch.Size([20000, 300]))

In [13]:
# Note: We don't actually use these words since the model doesn't care about them.
# We just compute them in case we want to check some particular word or something.
len(train_words), len(validation_words), len(test_words)

(360000, 20000, 20000)

In [55]:
train_dataloader = torch.utils.data.DataLoader(
    train_vectors, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu
)
validation_dataloader = torch.utils.data.DataLoader(
    validation_vectors, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu
)
test_dataloader = torch.utils.data.DataLoader(
    test_vectors, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu
)

# Model Time

In [56]:
class AutoEncoderWithoutHiddenLayer(nn.Module):
    def __init__(self, input_size, bottleneck_size):
        super().__init__()
        self.encoder = nn.Linear(in_features=input_size, out_features=bottleneck_size)
        self.encoder_activation = nn.Tanh()
        
        self.decoder = nn.Linear(in_features=bottleneck_size, out_features=input_size)
        self.decoder_activation = nn.Tanh()
        
        self.decoder.weight = nn.Parameter(self.encoder.weight.transpose(0,1))
    
    def forward(self, features):
        latent_representation = self.encoder_activation(self.encoder(features))
        reconstructed_input = self.decoder_activation(self.decoder(latent_representation))
        return reconstructed_input
    
    def encode(self, features):
        return self.encoder_activation(self.encoder(features))

In [57]:
class AutoEncoderWithHiddenLayer(nn.Module):
    def __init__(self, input_size, hidden_size, bottleneck_size):
        super().__init__()
        self.encoder_input = nn.Linear(in_features=input_size, out_features=hidden_size)
        self.encoder_input_activation = nn.ReLU(True)
        self.encoder_hidden = nn.Linear(in_features=hidden_size, out_features=bottleneck_size)
        self.encoder_hidden_activation = nn.Tanh()
        
        self.decoder_hidden = nn.Linear(in_features=bottleneck_size, out_features=hidden_size)
        self.decoder_hidden_activation = nn.ReLU(True)
        self.decoder_output = nn.Linear(in_features=hidden_size, out_features=input_size)
        self.decoder_output_activation = nn.Tanh()
        
        self.decoder_hidden.weight = nn.Parameter(self.encoder_hidden.weight.transpose(0,1))
        self.decoder_output.weight = nn.Parameter(self.encoder_input.weight.transpose(0,1))
        
        self.encoder = [
            self.encoder_input, self.encoder_input_activation, self.encoder_hidden, self.encoder_hidden_activation
        ]
        self.decoder = [
            self.decoder_hidden, self.decoder_hidden_activation, self.decoder_output, self.decoder_output_activation
        ]
    
    def forward(self, features):
        # Encoder
        for layer in self.encoder:
            features = layer(features)
        # Decoder
        for layer in self.decoder:
            features = layer(features)
        return features
    
    def encode(self, features):
        for layer in self.encoder:
            features = layer(features)
        return features

In [58]:
if use_hidden_layer:
    model = AutoEncoderWithHiddenLayer(input_size, hidden_size, bottleneck_size).to(device)
else:
    model = AutoEncoderWithoutHiddenLayer(input_size, bottleneck_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [59]:
model

AutoEncoderWithoutHiddenLayer(
  (encoder): Linear(in_features=300, out_features=200, bias=True)
  (encoder_activation): Tanh()
  (decoder): Linear(in_features=200, out_features=300, bias=True)
  (decoder_activation): Tanh()
)

In [60]:
for epoch in range(epochs):
    train_loss = 0
    validation_loss = 0
    
    # Training Loop
    for iteration, batch in enumerate(tqdm(train_dataloader)):
        # Reset gradients back to zero for this iteration
        optimizer.zero_grad()
        
        if denoising:
            # Add noise to our inputs
            noise = torch.randn(batch.shape)
            noisy_batch = torch.clamp(batch + noise, -1, +1)
            
            # Move batch to device
            noisy_batch = noisy_batch.to(device)

            # Run our model & get outputs
            outputs = model(noisy_batch)
        else:
            # Move batch to device
            batch = batch.to(device)

            # Run our model & get outputs
            outputs = model(batch)
            
        # Calculate reconstruction loss
        batch_loss = criterion(outputs, batch)
                  
        # Backprop
        batch_loss.backward()
        
        # Update our optimizer parameters
        optimizer.step()
        
        # Add the batch's loss to the total loss for the epoch
        train_loss += batch_loss.item()
        
    # Validation Loop
    with torch.no_grad():
        for iteration, batch in enumerate(tqdm(validation_dataloader)):
            # Move batch to device
            batch = batch.to(device)

            # Run our model & get outputs
            outputs = model(batch)

            # Calculate reconstruction loss
            batch_loss = criterion(outputs, batch)

            # Add the batch's loss to the total loss for the epoch
            validation_loss += batch_loss.item()
    
    # Compute the average losses for this epoch
    train_loss = train_loss / len(train_dataloader)
    validation_loss = validation_loss / len(validation_dataloader)
    
    # Print Metrics
    print(
        f"Epoch: {epoch+1}/{epochs}, Train Reconstruction Loss = {train_loss}, \
        Validation Reconstruction Loss = {validation_loss}"
    )

  0%|                                                                                         | 0/2813 [00:00<?, ?it/s]

Epoch: 1/5, Train Reconstruction Loss = 0.11352380897506802,         Validation Reconstruction Loss = 0.08517283495444401


  0%|                                                                                         | 0/2813 [00:00<?, ?it/s]

Epoch: 2/5, Train Reconstruction Loss = 0.11308554488698062,         Validation Reconstruction Loss = 0.08506235428107013


  0%|                                                                                         | 0/2813 [00:00<?, ?it/s]

Epoch: 3/5, Train Reconstruction Loss = 0.11305423933513385,         Validation Reconstruction Loss = 0.08547512783556227


  0%|                                                                                         | 0/2813 [00:00<?, ?it/s]

Epoch: 4/5, Train Reconstruction Loss = 0.11302302716583956,         Validation Reconstruction Loss = 0.084906534831615


100%|███████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 158.23it/s]

Epoch: 5/5, Train Reconstruction Loss = 0.11303879632894313,         Validation Reconstruction Loss = 0.08527829948883907


# Test Model

In [61]:
model.eval()

AutoEncoderWithoutHiddenLayer(
  (encoder): Linear(in_features=300, out_features=200, bias=True)
  (encoder_activation): Tanh()
  (decoder): Linear(in_features=200, out_features=300, bias=True)
  (decoder_activation): Tanh()
)

In [62]:
reconstruction_loss = 0

# Testing Loop
with torch.no_grad():
    for iteration, batch in enumerate(tqdm(test_dataloader)):
        # Move batch to device
        batch = batch.to(device)
        
        # Run our model & get outputs
        outputs = model(batch)

        # Calculate reconstruction loss
        batch_loss = criterion(outputs, batch)

        # Add the batch's loss to the total loss for the epoch
        reconstruction_loss += batch_loss.item()

# Compute the average losses for this epoch
reconstruction_loss = reconstruction_loss / len(test_dataloader)

# Print Metrics
print(
    f"Test Reconstruction Loss = {reconstruction_loss}"
)

100%|███████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 153.74it/s]

Test Reconstruction Loss = 0.08506599647603977


# Generate Latent Embeddings

In [63]:
latent_vectors = {}
with torch.no_grad():
    for i, (word, vector) in enumerate(tqdm(zip(words, vectors))):
        latent_vectors[word] = model.encode(vector)

400000it [00:19, 20004.54it/s]


In [64]:
len(latent_vectors)

400000

In [65]:
latent_vectors['the']

tensor([-5.9412e-02,  5.8080e-02,  7.8171e-02, -6.7203e-02, -1.9439e-01,
        -5.8522e-02,  1.9077e-01, -9.1857e-02, -2.6136e-02, -2.2030e-01,
         5.4154e-02,  1.1263e-01,  9.7602e-01,  2.1195e-01, -3.5939e-02,
         5.1026e-02,  9.1576e-02,  3.6884e-02,  2.2810e-02, -1.6704e-02,
        -9.7355e-02,  1.7112e-01,  7.1116e-02, -2.9789e-01, -9.5616e-04,
        -1.7079e-01, -1.2019e-01,  2.7641e-01, -9.1226e-03, -6.0054e-01,
        -9.7913e-02, -9.3085e-02, -2.3570e-01,  2.9613e-02,  5.1035e-02,
        -1.0977e-01, -2.1756e-02,  1.7778e-03, -4.1988e-02, -1.5805e-01,
        -1.3762e-01, -4.4904e-01, -7.5584e-02, -4.6367e-02, -8.8341e-02,
         4.2866e-02,  4.1189e-02, -1.1010e-02,  3.6080e-02,  3.3598e-02,
         4.3671e-02,  1.8183e-01,  9.3230e-02,  1.7387e-01,  4.2324e-02,
        -6.1247e-02,  1.9069e-01, -6.1776e-02, -1.1885e-01,  1.4381e-01,
        -2.8281e-01, -1.2407e-02, -1.9807e-01,  1.4388e-01, -2.4034e-01,
        -2.4540e-01, -1.3305e-01, -1.2210e-01, -1.0

# Save Model & Embeddings

In [66]:
torch.save(model.state_dict(), f"models/{experiment_name}.pt")

In [67]:
# Need to convert the latent embeddings into the glove format
# word dim1 dim2 dim3 dim4 ... dimX
lines = []
for i, (word, vector) in tqdm(enumerate(latent_vectors.items())):
    line = [word] + [str(x) for x in vector.tolist()]
    lines.append(' '.join(line))

400000it [00:58, 6865.85it/s]


In [68]:
with open(f"data/embeddings/trained/{experiment_name}.glove.6B.300d.txt", "w", encoding="utf-8") as fp:
    fp.write("\n".join(lines))